# Six Degrees of Kevin Bacon
**Introduction** - Six Degrees of Kevin Bacon is a game based on the "six degrees of separation"
concept, which posits that any two people on Earth are six or fewer acquaintance links apart. Movie
buffs challenge each other to find the shortest path between an arbitrary actor and prolific actor
Kevin Bacon. It rests on the assumption that anyone involved in the film industry can be linked
through their film roles to Bacon within six steps.
The analysis of social networks can be a computationally intensive task, especially when dealing with
large volumes of data. It is also a challenging problem to devise a correct methodology to infer an
informative social network structure. Here, we will analyze a social network of actors and actresses
that co-participated in movies. We will do some simple descriptive analysis, and in the end try to
relate an actor/actress’s position in the social network with the success of the movies in which they
participate.

####Rules & Notes - Please take your time to read the following points:
2. The submission deadline will be set for the 30th of May at 23:59h.
3. It is acceptable that you **discuss** with your colleagues different approaches to solve each step of the problem set, but the assignment is individual. That is, you are responsible for writing your own code, and analyzing the results. Clear cases of cheating will be penalized with 0 points in this assignment.
4. After review of your submission files, and before a mark is attributed, you might be called to orally defend your submission.
5. You will be scored first and foremost by the number of correct answers, secondly by the logic used in trying to approach each step of the problem set.
1. You can add as many cells as you like to answer the questions.
1. It is also important you clearly indicate what your final answer to each question is when you are using multiple cells (for example you can use `print("My final anwser is:")` before your answer or use cell comments).
11. Consider skipping questions that you are stuck on, and get back to them later.
12. Expect computations to take a few minutes to finish in some of the steps.
1. It is recommended you read the whole assignment before starting.
1. You can make use of caching or persisting your RDDs or Dataframes, this may speed up performance. You do not need to cache every dataframe, but usually you want to do this at least once after the data has been imported.
1. If you have trouble with graphframes in databricks (specifically the import statement) you need to make sure the graphframes package is installed on the cluster you are running. If you click home on the left, then click on the graphframes library which you loaded in Lab 11 you can install the package on your cluster (check the graphframes checkbox and click install)
1. Be careful, you must not 'Publish' this notebook in databricks.
13. **IMPORTANT** It is expected you have developed skills beyond writing SQL queries. Any question where you directly write a SQL query (by for example creating a temporary view and then using spark.sql to pass the query) will receive a 25% penalty. Using the spark syntax (for example dataframe.select("\*").where("conditions")) is acceptable and does not incur this penalty.
13. **Questions** – Any questions about this assignment should be posted in the Forum@Moodle. Questions by e-mail will not be answered. The lab will run at the normal time. During this period you can ask any questions you have about the exam (we can't provide you the actual answers of course, but there may be helpful tips if you are stuck on any of the steps). As such, it is probably useful to attempt the assignment before the scheduled lab.
14. **Delivery** - To fulfil this activity you will have to upload the following materials to Moodle:
 - An exported IPython notebook. From the menu at the top, select 'File', then 'Export', then 'IPython Notebook', to download the notebook. The notebook should be solved (have results displayed), but should contain all necessary code so that when the notebook is run in databricks it should also replicate these results. This means that all data downloading and processing should be done in this notebook. It is also important you clearly indicate where your final answer to each question is when you are using multiple cells (for example you can us `print("My final answer is:")` before your answer or use cell comments).
 - A PDF version of your code and answers. There are a couple of ways you can do this. You can convert the downloaded IPython Notebook to pdf (check out nbconvert if you have Jupyter notebook), or you can just copy your code and answers into a word file and save as pdf, or finally you can take screenshots of each page of the notebook and put them into a word file and save it as pdf. It is important that all code and aswers are visible in this pdf.
  - You will also need to provide a signed statement of authorship, which is available on Moodle.


####Data Sources and Description
We will use data from IMDB. You can download raw datafiles
from https://datasets.imdbws.com. Note that the files are tab delimited (.tsv) You can find a
description of the each datafile in https://www.imdb.com/interfaces/

## Questions
### Data loading and preperation
Review the file descriptions and load the necessary data into your databricks cluser and into spark dataframes or rdds. You will need to use shell commands to download the data, unzip the data, load the data into spark. Note that the data might require parsing and preprocessing to be ready for the questions below.

**Hints** You can use `gunzip` to unzip the .tz files. The data files will then be tab seperated (.tsv), which you can load into a dataframe using the tab seperated option instead of the comma seperated option we have typically used in class: `.option(“sep”,”\t”)`

**Libraries**

In [4]:
from pyspark import StorageLevel
from pyspark.sql.functions import when, lit, col
from pyspark.sql.types import *
from pyspark.sql import Row
from graphframes import *
import graphframes.graphframe as gfm
from pyspark.ml.feature import VectorAssembler, VectorIndexer

**Download gz files**

In [6]:
%sh 
wget --quiet https://datasets.imdbws.com/name.basics.tsv.gz
wget --quiet https://datasets.imdbws.com/title.basics.tsv.gz
wget --quiet https://datasets.imdbws.com/title.akas.tsv.gz
wget  --quiet https://datasets.imdbws.com/title.principals.tsv.gz
wget  --quiet https://datasets.imdbws.com/title.ratings.tsv.gz


**unzip**

In [8]:
%sh 
gunzip name.basics.tsv.gz
gunzip title.akas.tsv.gz
gunzip title.basics.tsv.gz
gunzip title.principals.tsv.gz
gunzip title.ratings.tsv.gz

**creating data frames**

In [10]:
df_name_basics      = spark.read.option("sep", "\t").csv('file:/databricks/driver/name.basics.tsv',      header=True, inferSchema = True)
df_title_akas       = spark.read.option("sep", "\t").csv('file:/databricks/driver/title.akas.tsv',    header=True, inferSchema = True)
df_title_basics     = spark.read.option("sep", "\t").csv('file:/databricks/driver/title.basics.tsv',     header=True, inferSchema = True)
df_title_principals = spark.read.option("sep", "\t").csv('file:/databricks/driver/title.principals.tsv', header=True, inferSchema = True)
df_title_ratings    = spark.read.option("sep", "\t").csv('file:/databricks/driver/title.ratings.tsv',    header=True, inferSchema = True)


In [11]:
df_name_basics.cache()
df_title_akas.cache()
df_title_basics.cache()    
df_title_principals.cache()
df_title_ratings.cache()  

Out[8]: DataFrame[tconst: string, averageRating: double, numVotes: int]

In [12]:
df_name_basics.show(4)    
df_title_akas.show(4)
df_title_basics.show(4)    
df_title_principals.show(4)
df_title_ratings.show(4)   

+---------+---------------+---------+---------+--------------------+--------------------+
 nconst| primaryName|birthYear|deathYear| primaryProfession| knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
nm0000001| Fred Astaire| 1899| 1987|soundtrack,actor,...|tt0043044,tt00531...|
nm0000002| Lauren Bacall| 1924| 2014| actress,soundtrack|tt0071877,tt01170...|
nm0000003|Brigitte Bardot| 1934| \N|actress,soundtrac...|tt0054452,tt00491...|
nm0000004| John Belushi| 1949| 1982|actor,soundtrack,...|tt0072562,tt00804...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 4 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
 titleId|ordering| title|region|language| types| attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
tt0000001| 1| Карменсіта| UA| \N|imdbDisplay| \N| 0|
tt0000001| 2| Carmencita| DE| \N| \N|literal title| 0|
tt0000001| 3|Carmencita - span...| HU| \N|imdbDisplay| \N| 0|
tt0000001| 4| Καρμενσίτα| GR| \N|imdbDisplay| \N| 0|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
only showing top 4 rows

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
 tconst|titleType| primaryTitle| originalTitle|isAdult|startYear|endYear|runtimeMinutes| genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
tt0000001| short| Carmencita| Carmencita| 0| 1894| \N| 1| Documentary,Short|
tt0000002| short|Le clown et ses c...|Le clown et ses c...| 0| 1892| \N| 5| Animation,Short|
tt0000003| short| Pauvre Pierrot| Pauvre Pierrot| 0| 1892| \N| 4|Animation,Comedy,...|
tt0000004| short| Un bon bock| Un bon bock| 0| 1892| \N| 12| Animation,Short|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
only showing top 4 rows

+---------+--------+---------+---------------+--------------------+----------+
 tconst|ordering| nconst| category| job|characters|
+---------+--------+---------+---------------+--------------------+----------+
tt0000001| 1|nm1588970| self| \N| ["Self"]|
tt0000001| 2|nm0005690| director| \N| \N|
tt0000001| 3|nm0374658|cinematographer|director of photo...| \N|
tt0000002| 1|nm0721526| director| \N| \N|
+---------+--------+---------+---------------+--------------------+----------+
only showing top 4 rows

+---------+-------------+--------+
 tconst|averageRating|numVotes|
+---------+-------------+--------+
tt0000001| 5.6| 1617|
tt0000002| 6.0| 198|
tt0000003| 6.5| 1299|
tt0000004| 6.1| 122|
+---------+-------------+--------+
only showing top 4 rows

### Network Inference, Let’s build a network
In the following questions you will look to summarise the data and build a network. We want to examine a network that abstracts how actors and actress are related through their co-participation in movies. To that end perform the following steps:

**Q1** Create a DataFrame that combines the information on each of the titles (i.e., movies, tv-shows, etc …) and the information on the participants in those movies (i.e., actors, directors, etc … ), make sure the actual names of the movies and participants are included. It may be worth reviewing the following questions to see how this dataframe will be used.

How many rows does yoru dataframe have?

In [14]:
df_network = df_title_principals.join(df_name_basics, on=['nconst'], how='left') \
                                     .join(df_title_basics, on=['tconst'], how='left')
df_network.show(5)
df_network.cache()

+---------+---------+--------+--------+--------+------------------+--------------------+---------+---------+--------------------+--------------------+---------+--------------------+--------------------+-------+---------+-------+--------------+---------------+
 tconst| nconst|ordering|category| job| characters| primaryName|birthYear|deathYear| primaryProfession| knownForTitles|titleType| primaryTitle| originalTitle|isAdult|startYear|endYear|runtimeMinutes| genres|
+---------+---------+--------+--------+--------+------------------+--------------------+---------+---------+--------------------+--------------------+---------+--------------------+--------------------+-------+---------+-------+--------------+---------------+
tt0000658|nm0169871| 1|director| \N| \N| Émile Cohl| 1857| 1938|director,writer,a...|tt1003497,tt10035...| short|The Puppet's Nigh...|Le cauchemar de F...| 0| 1908| \N| 2|Animation,Short|
tt0000839|nm0378408| 2|producer|producer| \N| Cecil M. Hepworth| 1873| 1953|cinematographer,p...|tt0000420,tt03082...| short| The Curse of Money| The Curse of Money| 0| 1909| \N| \N| Drama,Short|
tt0000839|nm0294276| 1|director| \N| \N| Theo Frenkel| 1871| 1956|director,actor,wr...|tt0006318,tt00112...| short| The Curse of Money| The Curse of Money| 0| 1909| \N| \N| Drama,Short|
tt0001170|nm0607104| 10| actor| \N| \N| Chick Morrison| 1878| 1924| actor|tt0011973,tt00050...| short|A Cowboy's Vindic...|A Cowboy's Vindic...| 0| 1910| \N| \N| Short,Western|
tt0001170|nm0001908| 1| actor| \N|["Frank Morrison"]|Gilbert M. 'Bronc...| 1880| 1971|director,actor,pr...|tt0183803,tt00024...| short|A Cowboy's Vindic...|A Cowboy's Vindic...| 0| 1910| \N| \N| Short,Western|
+---------+---------+--------+--------+--------+------------------+--------------------+---------+---------+--------------------+--------------------+---------+--------------------+--------------------+-------+---------+-------+--------------+---------------+
only showing top 5 rows

Out[6]: DataFrame[tconst: string, nconst: string, ordering: int, category: string, job: string, characters: string, primaryName: string, birthYear: string, deathYear: string, primaryProfession: string, knownForTitles: string, titleType: string, primaryTitle: string, originalTitle: string, isAdult: int, startYear: string, endYear: string, runtimeMinutes: string, genres: string]

In [15]:
df_network.count()

Out[7]: 39471189

**Q1 My final answer is :** 39.471.189

** Q2 ** Create a new DataFrame based on the previous step, with the following removed:
1. Any participant that is not an actor or actress (as measured by the category column);
1. All adult movies;
1. All dead actors or actresses;
1. All actors or actresses born before 1920 or with no date of birth listed;
1. All titles that are not of the type movie.

How many rows does your dataframe have?

In [18]:
def replace(column, value):
    return when(column != value, column).otherwise(lit(None))

df_network = df_network.withColumn("birthYear", replace(col("birthYear"), "\\N")) \
                                  .withColumn("deathYear", replace(col("deathYear"), "\\N"))


In [19]:
df_network_filter = df_network.filter(df_network.category.isin(['actor', 'actress'])) \
                                      .filter(df_network.isAdult != 1) \
                                      .filter(df_network.deathYear.isNull()) \
                                      .filter(df_network.birthYear.isNotNull()) \
                                      .filter(df_network.birthYear >= 1920) \
                                      .filter(df_network.titleType == 'movie')

df_network_filter.cache()
df_network_filter.show(4)

+---------+---------+--------+--------+---+----------------+----------------+---------+---------+--------------------+--------------------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
 tconst| nconst|ordering|category|job| characters| primaryName|birthYear|deathYear| primaryProfession| knownForTitles|titleType| primaryTitle| originalTitle|isAdult|startYear|endYear|runtimeMinutes| genres|
+---------+---------+--------+--------+---+----------------+----------------+---------+---------+--------------------+--------------------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
tt0036077|nm0788396| 2| actress| \N| ["Rani"]| Mumtaz Shanti| 1926| null| actress|tt0040387,tt03758...| movie| Kismet| Kismet| 0| 1943| \N| 143| Drama|
tt0036808|nm0936730| 1| actress| \N|["Mary Elliott"]| Jane Withers| 1926| null|actress,soundtrac...|tt0049261,tt00330...| movie| Faces in the Fog| Faces in the Fog| 0| 1944| \N| 71| Drama|
tt0037459|nm0527996| 1| actor| \N| ["Ted Benson"]| Jimmy Lydon| 1923| null|actor,producer,mi...|tt0038126,tt00348...| movie|When the Lights G...|When the Lights G...| 0| 1944| \N| 74|Drama,Romance,War|
tt0039955|nm0319855| 4| actress| \N| \N|Carolina Jiménez| 1922| null|actress,miscellan...|tt0339560,tt00416...| movie| Un viaje de novios| Un viaje de novios| 0| 1948| \N| 95| \N|
+---------+---------+--------+--------+---+----------------+----------------+---------+---------+--------------------+--------------------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
only showing top 4 rows

In [20]:
df_network_filter.count()

Out[10]: 451091

"**Q2 My final answer is :** 451.091

**Q3** Convert the above Dataframe to an RDD (you can use `.rdd` to convert a dataframe to and RDD of row objects). Use map and reduce to create a paired RDD which counts how many movies each actor / actress appears in.

Display names of the top 10 actors/actresses according to the number of movies in which they appeared. Be careful to deal with different actors / actresses with the same name, these could be different people.

In [23]:
rdd_network_filter = df_network_filter.rdd.map(list)
rdd_network_filter.take(4)

Out[11]: [['tt0036077',
 'nm0788396',
 2,
 'actress',
 '\\N',
 '["Rani"]',
 'Mumtaz Shanti',
 '1926',
 None,
 'actress',
 'tt0040387,tt0375805,tt0138315,tt0425763',
 'movie',
 'Kismet',
 'Kismet',
 0,
 '1943',
 '\\N',
 '143',
 'Drama'],
 ['tt0036808',
 'nm0936730',
 1,
 'actress',
 '\\N',
 '["Mary Elliott"]',
 'Jane Withers',
 '1926',
 None,
 'actress,soundtrack,writer',
 'tt0049261,tt0033044,tt0027667,tt0257778',
 'movie',
 'Faces in the Fog',
 'Faces in the Fog',
 0,
 '1944',
 '\\N',
 '71',
 'Drama'],
 ['tt0037459',
 'nm0527996',
 1,
 'actor',
 '\\N',
 '["Ted Benson"]',
 'Jimmy Lydon',
 '1923',
 None,
 'actor,producer,miscellaneous',
 'tt0038126,tt0034844,tt0039566,tt0075393',
 'movie',
 'When the Lights Go on Again',
 'When the Lights Go on Again',
 0,
 '1944',
 '\\N',
 '74',
 'Drama,Romance,War'],
 ['tt0039955',
 'nm0319855',
 4,
 'actress',
 '\\N',
 '\\N',
 'Carolina Jiménez',
 '1922',
 None,
 'actress,miscellaneous',
 'tt0339560,tt0041665,tt0203156,tt0043978',
 'movie',
 'Un viaje de novios',
 'Un viaje de novios',
 0,
 '1948',
 '\\N',
 '95',
 '\\N']]

In [24]:
rdd_network_id = rdd_network_filter.map(lambda x :(str(x[1]), 1))
rdd_network_name = rdd_network_filter.map(lambda x :(str(x[1]), str(x[6]))).distinct()

In [25]:
rdd_network_count = rdd_network_id.reduceByKey(lambda x,y: x+y)

In [26]:
final_count = rdd_network_count.join(rdd_network_name).sortBy(lambda x: x[1][0], False).map(lambda x :(x[0], x[1][1], x[1][0]))
final_count.take(10)

Out[14]: [('nm0103977', 'Brahmanandam', 809),
 ('nm0007123', 'Mammootty', 379),
 ('nm0482320', 'Mohanlal', 343),
 ('nm0149822', 'Mithun Chakraborty', 332),
 ('nm0007106', 'Shakti Kapoor', 309),
 ('nm0415549', 'Jagathi Sreekumar', 303),
 ('nm0035067', 'Cüneyt Arkin', 294),
 ('nm0374974', 'Helen', 281),
 ('nm0534867', 'Madhu', 277),
 ('nm0004429', 'Dharmendra', 270)]

**Q3 My final answer is :**

|    | nconst    | primaryName        |   qtt |
|---:|:----------|:-------------------|------:|
|  0 | nm0103977 | Brahmanandam       |   809 |
|  1 | nm0007123 | Mammootty          |   379 |
|  2 | nm0482320 | Mohanlal           |   343 |
|  3 | nm0149822 | Mithun Chakraborty |   332 |
|  4 | nm0007106 | Shakti Kapoor      |   309 |
|  5 | nm0415549 | Jagathi Sreekumar  |   303 |
|  6 | nm0035067 | Cüneyt Arkin       |   294 |
|  7 | nm0374974 | Helen              |   281 |
|  8 | nm0534867 | Madhu              |   277 |
|  9 | nm0004429 | Dharmendra         |   270 |

**Q4** Start with the dataframe from **Q2**. Generate a DataFrame that lists all links of your network. Here we shall consider that a link connects a pair of actors/actresses if they participated in at least one movie together (actors / actresses should be represented by their unique ID's). For every link we then need anytime a pair of actors were together in a movie as a link in each direction (A -> B and B -> A). However links should be distinct we do not need duplicates when two actors worked together in several movies.

In [29]:
rdd_actors_link1 = df_network_filter.rdd.map(list).map(lambda x :(x[0], x[1]))
rdd_actors_link2 = df_network_filter.rdd.map(list).map(lambda x :(x[0], x[1]))

In [30]:
rdd_actors_link =  rdd_actors_link1.join(rdd_actors_link2) \
                                        .map(lambda x :Row(x[1][0], x[1][1])) \
                                        .filter(lambda x: x[0] != x[1]).distinct() 
rdd_actors_link.take(3)

Out[16]: [<Row(nm0668271, nm0001151)>,
 <Row(nm2507102, nm0102403)>,
 <Row(nm3216408, nm0453304)>]

In [31]:
ActorA = StructField("ActorA",StringType(),True)
ActorB = StructField("ActorB",StringType(),True)

df_actors_link = sqlContext.createDataFrame(rdd_actors_link, StructType([ActorA, ActorB]))
df_actors_link.cache()
df_actors_link.show(5)
df_actors_link.count()

+---------+---------+
 ActorA| ActorB|
+---------+---------+
nm0668271|nm0001151|
nm2507102|nm0102403|
nm3216408|nm0453304|
nm0429385|nm0005541|
nm0544425|nm0000778|
+---------+---------+
only showing top 5 rows

Out[17]: 712112

**Q4 My final answer is :**

|   ActorA|   ActorB|
|---------|---------|
|nm0668271|nm0001151|
|nm2507102|nm0102403|
|nm3216408|nm0453304|
|nm0429385|nm0005541|
|nm0544425|nm0000778|

only the first 5 rows of 712.112 rows total

**Q5** Compute the page rank of each actor. This can be done using GraphFrames or
by using RDDs and the iterative implementation of the PageRank algorithm. Do not take
more than 5 iterations and use reset probility = 0.1.

List the top 10 actors / actresses by pagerank.

In [34]:
df_vertices = df_actors_link.select(df_actors_link['ActorA']).selectExpr("ActorA as id").distinct()
df_link = df_actors_link.selectExpr("ActorA as src", "ActorB as dst")

In [35]:
ourGraph = gfm.GraphFrame(df_vertices, df_link)
ourGraph.vertices.show(4)
ourGraph.edges.show(4)

+---------+
 id|
+---------+
nm0527543|
nm1699244|
nm0005351|
nm1163128|
+---------+
only showing top 4 rows

+---------+---------+
 src| dst|
+---------+---------+
nm0668271|nm0001151|
nm2507102|nm0102403|
nm3216408|nm0453304|
nm0429385|nm0005541|
+---------+---------+
only showing top 4 rows

In [36]:
pageRanks = ourGraph.pageRank(resetProbability=0.1, maxIter = 5)

In [37]:
df_pageRank = pageRanks.vertices
df_pageRankTop10 = df_pageRank.sort('pageRank', ascending=False).limit(10)


In [38]:
df_pageRankTop10 = df_pageRankTop10.join(df_name_basics, df_pageRankTop10.id == df_name_basics.nconst,how='left') \
                                        .select(df_pageRankTop10['id'],
                                                df_name_basics['primaryName'],
                                                df_pageRankTop10['pagerank'])

df_pageRankTop10.sort('pageRank', ascending=False).show()

+---------+----------------+------------------+
 id| primaryName| pagerank|
+---------+----------------+------------------+
nm0000616| Eric Roberts| 40.6478874245532|
nm0000514| Michael Madsen|24.226138058070163|
nm0001744| Tom Sizemore|23.419848197831612|
nm0001803| Danny Trejo|20.555441795470156|
nm0000448| Lance Henriksen|17.663207647023903|
nm0001698| John Savage|17.583580336085777|
nm0004193| Debbie Rochon|16.881922296919786|
nm0000367|Gérard Depardieu|16.832482667099892|
nm0000800| Armand Assante|16.111563992409874|
nm0626259| Franco Nero|15.962778479477862|
+---------+----------------+------------------+

**Q5 My final answer is :**

|       id|     primaryName|          pagerank|
|---------|----------------|------------------|
|nm0000616|    Eric Roberts| 40.64568033439618|
|nm0000514|  Michael Madsen|24.223364243080432|
|nm0001744|    Tom Sizemore| 23.41212013915763|
|nm0001803|     Danny Trejo|  20.5416537452636|
|nm0000448| Lance Henriksen|17.663051148448265|
|nm0001698|     John Savage| 17.58626839735639|
|nm0004193|   Debbie Rochon| 16.84310918060613|
|nm0000367|Gérard Depardieu|16.831408391863988|
|nm0000800|  Armand Assante|16.109331083009028|
|nm0626259|     Franco Nero|15.963431437176633|

**Q6**: Create an RDD with the number of outDegrees for each actor. Display the top 10 by outDegrees.

In [41]:
df_outDegree = ourGraph.outDegrees
df_outDegreeTop10 = ourGraph.outDegrees.sort('outDegree', ascending=False).limit(10)
df_outDegreeTop10 = df_outDegreeTop10.join(df_name_basics, df_outDegreeTop10.id == df_name_basics.nconst,how='left').select(df_outDegreeTop10['id'], df_name_basics['primaryName'], df_outDegreeTop10['outDegree'])

df_outDegreeTop10.sort('outDegree', ascending=False).show()

+---------+--------------------+---------+
 id| primaryName|outDegree|
+---------+--------------------+---------+
nm0000616| Eric Roberts| 438|
nm0000514| Michael Madsen| 289|
nm0000367| Gérard Depardieu| 263|
nm0001744| Tom Sizemore| 261|
nm0945189| Simon Yam| 253|
nm0451600| Anupam Kher| 239|
nm0149822| Mithun Chakraborty| 232|
nm0001803| Danny Trejo| 231|
nm0874676| Eric Tsang| 227|
nm0938893|Anthony Chau-Sang...| 225|
+---------+--------------------+---------+

**Q6 My final answer is :**

|       id|         primaryName|outDegree|
|---------|--------------------|---------|
|nm0000616|        Eric Roberts|      438|
|nm0000514|      Michael Madsen|      289|
|nm0000367|    Gérard Depardieu|      263|
|nm0001744|        Tom Sizemore|      261|
|nm0945189|           Simon Yam|      253|
|nm0451600|         Anupam Kher|      238|
|nm0149822|  Mithun Chakraborty|      232|
|nm0001803|         Danny Trejo|      231|
|nm0874676|          Eric Tsang|      227|
|nm0938893|Anthony Chau-Sang...|      225|

### Let’s play Kevin’s own game

**Q7** Start with the graphframe / dataframe you developed in the previous section. Using Spark GraphFrame and/or Spark Core library perform the following steps:

1. Identify the id of Kevin Bacon, there are two actors named ‘Kevin Bacon’, we will use the one with the highest degree, that is, the one that participated in most titles;
1. Estimate the shortest path between every actor/actress in the database and Kevin Bacon, keep a dataframe with a column that includes the number of steps to Kevin Bacon as you will need it later (this will require a little processing to get from the graphframes output);
1. Summarise the data, that is, count the number of actors at each number of degress from kevin bacon (you will need to deal with actors unconnected to kevin bacon, if not connected to Kevin Bacon given these actors / actresses a score of 20). You could use the display() barchart functionality of databricks to easily display the distribution of the data.

*Note: The solution time on this step can be ~15 minutes*

In [44]:
df_name_basics.filter(df_name_basics.primaryName == 'Kevin Bacon').filter(df_name_basics.birthYear != '\\N').show()

+---------+-----------+---------+---------+--------------------+--------------------+
 nconst|primaryName|birthYear|deathYear| primaryProfession| knownForTitles|
+---------+-----------+---------+---------+--------------------+--------------------+
nm0000102|Kevin Bacon| 1958| \N|actor,producer,so...|tt0327056,tt03611...|
+---------+-----------+---------+---------+--------------------+--------------------+

In [45]:
df_Paths = ourGraph.shortestPaths(landmarks=["nm0000102"])

In [46]:
from pyspark.sql.functions import explode
df_shortestPaths = df_Paths.select("id", explode("distances"))  # explode("distances") this will become "value"
df_shortestPaths = df_shortestPaths.withColumn('value', when(df_shortestPaths.value.isNull(), lit(20)).otherwise(df_shortestPaths.value))
group_distances = df_shortestPaths.groupBy('value').count().sort('value')

In [47]:
df_shortestPaths.show(5)
df_shortestPaths.count()

+---------+---------+-----+
 id| key|value|
+---------+---------+-----+
nm0057741|nm0000102| 4|
nm0068551|nm0000102| 3|
nm0309307|nm0000102| 3|
nm2341870|nm0000102| 4|
nm0067745|nm0000102| 4|
+---------+---------+-----+
only showing top 5 rows

Out[27]: 66530

In [48]:
display(group_distances)

value,count
0,1
1,126
2,3303
3,18309
4,29051
5,13099
6,2251
7,311
8,38
9,11


**Q7.1 My final answer is: ** nm0000102
    
    
**Q7.2 My final answer is: **
    
|       id|      key|value|
|---------|---------|-----|
|nm0057741|nm0000102|    4|
|nm0068551|nm0000102|    3|
|nm0309307|nm0000102|    3|
|nm2341870|nm0000102|    4|
|nm0067745|nm0000102|    4|  
 
top 5 rows of, complete table has 66.516

**Q7.3 My final answer is: ** 

|value|count|
|-----|-----|
|    0|    1|
|    1|  126|
|    2| 3303|
|    3|18309|
|    4|29051|
|    5|13099|
|    6| 2251|
|    7|  311|
|    8|   38|
|    9|   11|
|   10|   12|
|   11|    4|

### Exploring the data with RDD's

Using RDDs and (not dataframes) answer the following questions (if you loaded your data into spark in a dataframe you can convert to an RDD of rows easily using `.rdd`) : 

*Hint:* paired RDD's will be useful.

**Q8** Movies can have multiple genres. Considering only titles of the type 'movie' what is the combination of genres that is the most popluar (as measured by number of reviews)?

In [51]:
rdd_genre   = df_network_filter.rdd.map(lambda x : (x[0], x[-1])).distinct()
rdd_reviews = df_title_ratings.rdd.map(lambda x : (x[0], x[-1]))
rdd_genreReviews = rdd_genre.join(rdd_reviews).map(lambda x : (x[1][0], x[1][1]))
rdd_genreReviews.take(5)

Out[28]: [('Drama', 8),
 ('Documentary', 11),
 ('Drama,War', 8955),
 ('Adventure,Drama', 75),
 ('Action,History,Romance', 90)]

In [52]:
genreMostReviews = rdd_genreReviews.reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1], False).take(1)
genreMostReviews

Out[29]: [('Action,Adventure,Sci-Fi', 43370766)]

** Q8 my final answer is **

| genre                   | qtt     |
|-------------------------|---------|
| Action,Adventure,Sci-Fi | 43364010|

**Q9** Movies can have multiple genres. Considering only titles of the type 'movie', and movies with more than 500 ratings, what is the combination of genres that has the highest **average movie rating** (you can average the movie rating for each movie in that genre combination).

In [55]:
rdd_reviews_gt500 = rdd_reviews.filter(lambda x: x[1] > 500)
rdd_ratings = df_title_ratings.rdd.map(lambda x : (x[0], x[1]))
rdd_movieRate = rdd_reviews_gt500.join(rdd_ratings).map(lambda x : (x[0], x[1][1]))
rdd_movieRate.take(5)

Out[30]: [('tt0000010', 6.9),
 ('tt0000022', 5.1),
 ('tt0000023', 5.7),
 ('tt0000041', 6.6),
 ('tt0000309', 6.1)]

In [56]:
rdd_genreRate = rdd_movieRate.join(rdd_genre).map(lambda x : (x[1][1], x[1][0]))
rdd_genreRate.take(5)

Out[31]: [('Comedy,Family,Fantasy', 7.1),
 ('Action,Sci-Fi,Thriller', 6.7),
 ('Comedy', 7.8),
 ('Action,Adventure,Drama', 6.5),
 ('Drama', 6.7)]

In [57]:
rdd_avgRating = rdd_genreRate.mapValues(lambda x: (x, 1)) \
                                       .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
                                       .mapValues(lambda x: x[0]/x[1]) \
                                       .sortBy(lambda x: x[1], False)
rdd_avgRating.take(5)

Out[32]: [('Comedy,Family,Mystery', 8.9),
 ('Music,Musical', 8.5),
 ('Drama,Fantasy,War', 8.2),
 ('Adventure,Drama,Musical', 8.1),
 ('Family,Musical', 8.0)]

** Q9 my final answer is **

| genre                   |avg movie rating|
|-------------------------|----------------|
| Comedy,Family,Mystery   |      8.9       |

**Q10** Movies can have multiple genres. What is **the individual genre** which is the most popular as meaured by number of votes. Votes for multiple genres count towards each genre listed.

*Hint: Think about the wordcount exercise we have done with RDDs.*

In [60]:
rdd_genreReviews.take(5)

Out[33]: [('Drama', 8),
 ('Documentary', 11),
 ('Drama,War', 8955),
 ('Adventure,Drama', 75),
 ('Action,History,Romance', 90)]

In [61]:
rdd_individualGenreReviews = rdd_genreReviews.map(lambda x : (x[0].split(','), x[1])) \
                                                 .map(lambda x: [(y, x[1]) for y in x[0]]) \
                                                 .flatMap(lambda x: x) \
                                                 .reduceByKey(lambda x, y: x+y) \
                                                 .sortBy(lambda x: x[1], False)
rdd_individualGenreReviews.take(4)

Out[34]: [('Drama', 382498197),
 ('Action', 252205415),
 ('Comedy', 233545282),
 ('Adventure', 213859699)]

** Q10 my final answer is: **

| genre   |qtt         |
|---------|----  ------|
| Drama   |382.388.240 |

## Engineering the perfect cast
We have created a number of potential features for predicting the rating of a movie based on its cast. Use sparkML to build a simple linear model to predict the rating of a movie based on the following features:

1. The total number of movies in which the actors / actresses in the current movie have acted (based on Q3)
1. The average pagerank of the cast in each movie (based on Q5)
1. The average outDegree of the cast in each movie (based on Q6)
1. The average value for for the cast of degrees of Kevin Bacon (based on Q7).

*If you were unable to generate any of these features as you could not answer the previous questions, just skip that particular feature.*

You will need to create a dataframe with the required features and label. Use a pipeline to create the vectors required by sparkML and apply the model. Remember to split your dataset, leave 30% of the data for testing, when splitting your data use the option `seed=0`.

**Q11** Provide the coefficients of the regression and the accuracy of your model on the test dataset according to RSME.

In [64]:
ActorId = StructField("ActorId",StringType(),True)
qtt_movies = StructField("qtt_movies",IntegerType(),True)
df_ActorMovies = sqlContext.createDataFrame(rdd_network_count, StructType([ActorId, qtt_movies]))

df_ActorsRates = df_ActorMovies.join(df_pageRank , df_ActorMovies.ActorId == df_pageRank.id , how = 'left') \
                                    .join(df_outDegree , df_ActorMovies.ActorId == df_outDegree.id , how = 'left') \
                                    .join(df_shortestPaths , df_ActorMovies.ActorId == df_shortestPaths.id , how = 'left')

In [65]:
df_ActorsRates = df_ActorsRates.select('ActorId', 'qtt_movies', 'pagerank', 'outDegree', 'value') \
                                    .withColumn('value', when(df_ActorsRates.value.isNull(), lit(20)).otherwise(df_ActorsRates.value))
df_ActorsRates.cache()
df_ActorsRates.show(6)

+---------+----------+-------------------+---------+-----+
 ActorId|qtt_movies| pagerank|outDegree|value|
+---------+----------+-------------------+---------+-----+
nm0000198| 51| 7.291971771730861| 122| 1|
nm0000354| 58| 6.26905749935267| 106| 2|
nm0000362| 47| 6.872262141837611| 108| 2|
nm0000767| 47| 7.025115732405378| 105| 3|
nm0002222| 2| 0.9981933151503921| 4| 4|
nm0003678| 1|0.20547451771110728| 1| 5|
+---------+----------+-------------------+---------+-----+
only showing top 6 rows

In [66]:
rdd_base = rdd_network_filter.map(lambda x: (x[0],x[1]))

tconst = StructField("tconst",StringType(),True)
nconst = StructField("nconst",StringType(),True)
df_base= sqlContext.createDataFrame(rdd_base, StructType([tconst, nconst]))

DF_aux_1 = df_base.join(df_title_ratings, on=['tconst'] ,how = 'left') \
                       .select(df_base['tconst'], df_base['nconst'], df_title_ratings['averageRating']) \
                       .join(df_ActorsRates,  df_base.nconst == df_ActorsRates.ActorId , how = 'inner')

DF_aux_1.cache()

Out[70]: DataFrame[tconst: string, nconst: string, averageRating: double, ActorId: string, qtt_movies: int, pagerank: double, outDegree: int, value: int]

In [67]:
from pyspark.sql import functions as F
DF_aux_1 = DF_aux_1.groupBy('tconst').agg(F.max('averageRating').alias('averageRating'), 
                                          F.sum('qtt_movies').alias('qtt_movies'), 
                                          F.avg('pagerank').alias('avg_pagerank'), 
                                          F.avg('outDegree').alias('avg_outDegree'), 
                                          F.avg('value').alias('avg_KB_Distance'))

In [68]:
DF_model = DF_aux_1.withColumn('avg_pagerank', when(DF_aux_1.avg_pagerank.isNull(), lit(0)).otherwise(DF_aux_1.avg_pagerank)) \
                              .withColumn('avg_outDegree', when(DF_aux_1.avg_outDegree.isNull(), lit(0)).otherwise(DF_aux_1.avg_outDegree)) \
                              .select('tconst', 'qtt_movies', 'avg_pagerank', 'avg_outDegree', 'avg_KB_Distance', 'averageRating') \
                              .distinct()

DF_model = DF_model.na.drop(subset=["averageRating"])
DF_model.cache()
DF_model.show(5)

+---------+----------+-------------------+-------------+---------------+-------------+
 tconst|qtt_movies| avg_pagerank|avg_outDegree|avg_KB_Distance|averageRating|
+---------+----------+-------------------+-------------+---------------+-------------+
tt0036077| 17|0.16915840003949434| 2.0| 4.0| 7.4|
tt0036808| 34|0.38807748292998767| 1.0| 5.0| 5.9|
tt0037459| 36| 1.3504494641289115| 3.0| 4.0| 5.6|
tt0040483| 1| 0.0| 0.0| 20.0| 5.7|
tt0042303| 67| 1.7001478487172963| 15.0| 3.0| 7.0|
+---------+----------+-------------------+-------------+---------------+-------------+
only showing top 5 rows

In [69]:
train, test = DF_model.randomSplit([0.7, 0.3], seed = 0)
train.show(5)

+---------+----------+-------------------+-------------+---------------+-------------+
 tconst|qtt_movies| avg_pagerank|avg_outDegree|avg_KB_Distance|averageRating|
+---------+----------+-------------------+-------------+---------------+-------------+
tt0036808| 34|0.38807748292998767| 1.0| 5.0| 5.9|
tt0037459| 36| 1.3504494641289115| 3.0| 4.0| 5.6|
tt0044770| 20| 1.023272944065177| 3.5| 3.5| 6.6|
tt0048006| 79| 1.7447342354032203| 16.0| 3.0| 5.9|
tt0049309| 45| 1.8702701327456883| 17.0| 4.0| 6.0|
+---------+----------+-------------------+-------------+---------------+-------------+
only showing top 5 rows

In [70]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

def transData(data):
  # Combine columns to a dense vector (excluding the last column)
  dataFeaturesRDD = data.rdd.map(lambda r: [Vectors.dense(r[1:-1]),r[-1]])
  
  # Convert the RDD back to a DataFrame, labelling the columns
  featuresDF =  dataFeaturesRDD.toDF(['features','label'])
  
  return featuresDF

In [71]:
train_set = transData(train)
test_set = transData(test)
train_set.show(4)

+--------------------+-----+
 features|label|
+--------------------+-----+
[34.0,0.388077482...| 5.9|
[36.0,1.350449464...| 5.6|
[20.0,1.023272944...| 6.6|
[79.0,1.744734235...| 5.9|
+--------------------+-----+
only showing top 4 rows

In [72]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression()
Model01 = lr.fit(train_set, {lr.regParam:0.3})

In [73]:
print(">>>> intercept: %r, beta qtt_movies : %r, beta avg_pagerank : %r, beta avg_outDegree : %r, beta avg_KB_Distance : %r" % (round(Model01.intercept, 4), round(Model01.coefficients[0], 4), round(Model01.coefficients[1], 4), round(Model01.coefficients[2],4), round(Model01.coefficients[3],4)))

>>>> intercept: 5.8888, beta qtt_movies : 0.0001, beta avg_pagerank : -0.0271, beta avg_outDegree : 0.0022, beta avg_KB_Distance : 0.0099

In [74]:
from pyspark.ml.evaluation import RegressionEvaluator

pred_test = Model01.transform(test_set)
evaluator = RegressionEvaluator(metricName="rmse")
RMSE_01 = evaluator.evaluate(pred_test)

print("Model01: RMSE = " + str(round(RMSE_01,4)))

Model01: RMSE = 1.2731

**Q11 My final answer is :**<br/>
**RMSE** = 1.2723 <br/>
**Coeficientes** = [ 5.8895, 0.0001, -0.0275, 0.0022, 0.0098 ]

**Q12** What score would your model predict for the 1997 movie Titanic and how does this compare to it's actual score.

In [77]:
df_title_basics.filter(df_title_basics.primaryTitle == 'Titanic').filter(df_title_basics.startYear == 1997).filter(df_title_basics.titleType == 'movie').show(5)

+---------+---------+------------+-------------+-------+---------+-------+--------------+-------------+
 tconst|titleType|primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes| genres|
+---------+---------+------------+-------------+-------+---------+-------+--------------+-------------+
tt0120338| movie| Titanic| Titanic| 0| 1997| \N| 194|Drama,Romance|
+---------+---------+------------+-------------+-------+---------+-------+--------------+-------------+

In [78]:
Titanic = DF_model.filter(DF_model.tconst == 'tt0120338')
Titanic = transData(Titanic)
pred_Titanic = Model01.transform(Titanic)
pred_Titanic.show()

+--------------------+-----+-----------------+
 features|label| prediction|
+--------------------+-----+-----------------+
[195.0,7.84596472...| 7.8|5.950696458127591|
+--------------------+-----+-----------------+

**Q12 My final answer is :**<br/>
**Predicted Score** = 5.95 <br/>
**Actual Score** = 7.8 <br/>
**Error** = 1.85

**Q13** Create dummy variables for each of the top 10 movie genres from **Q10**. These variable should have a value of 1 if the movie was rated with that genre and 0 otherwise. For example the 1997 movie Titanic should have a 1 in the dummy variable column for Romance, and a 1 in the dummy variable column for Drama, and 0's in all the other dummy variable columns.

*If you were unable to answer Q10 you can just select 10 different genres and construct the same data.*

*Note: Question 10 uses the number of votes per genre and not the average votes per genre.*

Does adding these variables to the regression improve your results? What is the new RMSE and predicted rating for the 1997 movie Titanic.

In [81]:
rdd_movie_genre = rdd_network_filter.map(lambda x: (x[-1],x[0]))
rdd_movie_genre = rdd_movie_genre.map(lambda x : (x[0].split(','), x[1])).map(lambda x: [(y, x[1]) for y in x[0]]).flatMap(lambda x: x)
rdd_movie_genre = rdd_movie_genre.map(lambda x : (x[1], x[0]))

tconst = StructField("movieId",StringType(),True)
genre = StructField("genre",StringType(),True)
dummy_movie_genre = sqlContext.createDataFrame(rdd_movie_genre, StructType([tconst, genre])).withColumn("genre", replace(col("genre"), "\\N"))

genre = rdd_individualGenreReviews.map(lambda x: x[0]).take(10)

dummy_movie_genre = dummy_movie_genre.filter(dummy_movie_genre.genre.isin(genre)).filter(dummy_movie_genre.genre.isNotNull())

dummy_movie_genre = dummy_movie_genre.groupBy("movieId").pivot("genre").agg(F.lit(1)).na.fill(0)
dummy_movie_genre.cache()
dummy_movie_genre.show(4)

+---------+------+---------+------+-----+-----+-------+-------+-------+------+--------+
 movieId|Action|Adventure|Comedy|Crime|Drama|Fantasy|Mystery|Romance|Sci-Fi|Thriller|
+---------+------+---------+------+-----+-----+-------+-------+-------+------+--------+
tt0054289| 0| 1| 0| 0| 1| 0| 0| 1| 0| 0|
tt0074792| 0| 0| 0| 1| 1| 0| 0| 0| 0| 1|
tt0084598| 1| 1| 0| 0| 0| 0| 0| 0| 0| 0|
tt0089167| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0|
+---------+------+---------+------+-----+-----+-------+-------+-------+------+--------+
only showing top 4 rows

In [82]:
DF_model_dummy = DF_model.join(dummy_movie_genre, DF_model.tconst == dummy_movie_genre.movieId, how = 'left') \
                              .drop('movieId').na.fill(0) \
                              .select('tconst', 'qtt_movies', 'avg_pagerank', 'avg_outDegree', 'avg_KB_Distance',
                                      'Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'Mystery', 
                                      'Romance', 'Sci-Fi', 'Thriller', 'averageRating')
                              
DF_model_dummy
DF_model_dummy.cache()
DF_model_dummy.show(4)

+---------+----------+-------------------+-------------+---------------+------+---------+------+-----+-----+-------+-------+-------+------+--------+-------------+
 tconst|qtt_movies| avg_pagerank|avg_outDegree|avg_KB_Distance|Action|Adventure|Comedy|Crime|Drama|Fantasy|Mystery|Romance|Sci-Fi|Thriller|averageRating|
+---------+----------+-------------------+-------------+---------------+------+---------+------+-----+-----+-------+-------+-------+------+--------+-------------+
tt0036077| 17|0.16915840003949434| 2.0| 4.0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 7.4|
tt0036808| 34|0.38807748292998767| 1.0| 5.0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 5.9|
tt0037459| 36| 1.3504494641289115| 3.0| 4.0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 5.6|
tt0040483| 1| 0.0| 0.0| 20.0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 5.7|
+---------+----------+-------------------+-------------+---------------+------+---------+------+-----+-----+-------+-------+-------+------+--------+-------------+
only showing top 4 rows

**Pipeline**

In [84]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

featuresCols = DF_model_dummy.columns
featuresCols.remove('averageRating')
featuresCols.remove('tconst')

vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=2)

lr = LinearRegression(labelCol="averageRating")

evaluator = RegressionEvaluator(metricName="rmse", labelCol=lr.getLabelCol(), predictionCol=lr.getPredictionCol())

grid = ParamGridBuilder().addGrid(lr.regParam, [0.01]) \
                                 .addGrid(lr.elasticNetParam, [0.25, 0.75]) \
                                 .addGrid(lr.maxIter, [5]) \
                                 .build()

cv = CrossValidator(estimator=lr, evaluator=evaluator, estimatorParamMaps = grid) #,

In [85]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])

In [86]:
train_dummy01, test_dummy01 = DF_model_dummy.randomSplit([0.7, 0.3], seed = 0)

In [87]:
pipelineModel = pipeline.fit(train_dummy01.drop('tconst'))

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [88]:
predictions = pipelineModel.transform(test_dummy01.drop('tconst'))

In [89]:
evaluator = RegressionEvaluator(metricName="rmse")
RMSE_02 = evaluator.evaluate(predictions.selectExpr('features', 'averageRating as label', 'prediction'))
print("Model03: RMSE = " + str(round(RMSE_02, 4)))

Model03: RMSE = 1.2347

In [90]:
Titanic_dummy = DF_model_dummy.filter(DF_model_dummy.tconst == 'tt0120338')
pred_Titanic_dummy = pipelineModel.transform(Titanic_dummy.drop('tconst'))
pred_Titanic_dummy.selectExpr('features', 'averageRating as label', 'prediction').show(4)

+--------------------+-----+-----------------+
 features|label| prediction|
+--------------------+-----+-----------------+
(14,[0,1,2,3,8,11...| 7.8|6.312599670636648|
+--------------------+-----+-----------------+

**Q13 My final answer is :**
The RMSE had a improvement of 3% from 1.27 to 1.23. For Titanic Predicted Score = 6.3 and Actual Score = 7.8, the error decrease to 1.5

**Q14** Improve your model by testing different machine learning algorithms, using hyperparameter tuning on these algorithms, changing the included features. Be careful not to cheat and use test data in the training of your model.

*Note: We are not testing your knowledge of different algorithms, we are just testing that you can apply the different tools in the spark toolkit and can compare between them.*

What is the RMSE of you final model and what rating does it predict for the 1997 movie Titanic.

In [94]:
rdd_reviews_gt500 = rdd_reviews.filter(lambda x: x[1] > 500)

rdd_ratings = df_title_ratings.rdd.map(lambda x : (x[0], x[1]))
rdd_movieActors = rdd_network_filter.map(lambda x : (x[0], x[1]))

rdd_movieActorRate = rdd_reviews_gt500.join(rdd_ratings).join(rdd_movieActors)
rdd_actorRate = rdd_movieActorRate.map(lambda x : (x[1][1], x[1][0][1]))

In [95]:
rdd_actorsAvgRating = rdd_actorRate.mapValues(lambda x: (x, 1)) \
                                       .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
                                       .mapValues(lambda x: x[0]/x[1]) \
                                       .sortBy(lambda x: x[1], False)
rdd_actorsAvgRating.take(5)

Out[170]: [('nm0637748', 9.9),
 ('nm1373275', 9.5),
 ('nm1617220', 9.5),
 ('nm5584750', 9.1),
 ('nm4485694', 9.1)]

In [96]:
rdd_actors_gt10Movies = rdd_network_count.filter(lambda x: x[1] > 5)
rdd_actorsAvgRating_b = rdd_actorsAvgRating.join(rdd_actors_gt10Movies) \
                                                .sortBy(lambda x: x[1][0], False) \
                                                .map(lambda x: (x[0],x[1][0])) \
                                                .filter(lambda x: x[1] > 7)\


rdd_actorsAvgRating_b.take(10)

Out[175]: [('nm1617220', 9.5),
 ('nm4485694', 9.1),
 ('nm0654744', 9.0),
 ('nm0153973', 9.0),
 ('nm0671635', 9.0),
 ('nm4468244', 8.9),
 ('nm0913307', 8.9),
 ('nm0791541', 8.9),
 ('nm0033175', 8.850000000000001),
 ('nm0620241', 8.8)]

In [97]:
rdd_aux = rdd_movieActors.map(lambda x : (x[1], x[0]))\
                             .join(rdd_actorsAvgRating_b)\
                             .map(lambda x : (x[1][0], 1))\
                             .reduceByKey(lambda x, y: x + y)
rdd_aux.take(4)

Out[176]: [('tt8959144', 1), ('tt0315817', 1), ('tt5543056', 1), ('tt0282914', 1)]

In [98]:
tconst = StructField("movieId",StringType(),True)
qttAvgRateGt8 = StructField("qttAvgRateGt8",IntegerType(),True)
df_movieActorsGt8 = sqlContext.createDataFrame(rdd_aux, StructType([tconst, qttAvgRateGt8]))
df_movieActorsGt8.sort('qttAvgRateGt8', ascending=False).show(4)

+---------+-------------+
 movieId|qttAvgRateGt8|
+---------+-------------+
tt7961060| 7|
tt7994100| 6|
tt6419510| 6|
tt3689894| 6|
+---------+-------------+
only showing top 4 rows

In [99]:
df_movieActorsGt8.count()

Out[178]: 35447

In [100]:
DF_model_actorsgt8 = DF_model_dummy.join(df_movieActorsGt8, DF_model_dummy.tconst == df_movieActorsGt8.movieId, how = 'left') \
                                        .drop('movieId').na.fill(0) \
                                        .select('tconst', 'qtt_movies', 'avg_pagerank', 'avg_outDegree', 'avg_KB_Distance',
                                                'Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'Mystery', 
                                                'Romance', 'Sci-Fi', 'Thriller','qttAvgRateGt8', 'averageRating')
DF_model_actorsgt8.cache()
DF_model_actorsgt8.show(5)


+---------+----------+-------------------+-------------+---------------+------+---------+------+-----+-----+-------+-------+-------+------+--------+-------------+-------------+
 tconst|qtt_movies| avg_pagerank|avg_outDegree|avg_KB_Distance|Action|Adventure|Comedy|Crime|Drama|Fantasy|Mystery|Romance|Sci-Fi|Thriller|qttAvgRateGt8|averageRating|
+---------+----------+-------------------+-------------+---------------+------+---------+------+-----+-----+-------+-------+-------+------+--------+-------------+-------------+
tt0036077| 17|0.16915840003949434| 2.0| 4.0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 7.4|
tt0036808| 34|0.38807748292998767| 1.0| 5.0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 5.9|
tt0037459| 36| 1.3504494641289115| 3.0| 4.0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 5.6|
tt0040483| 1| 0.0| 0.0| 20.0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 5.7|
tt0042303| 67| 1.7001478487172963| 15.0| 3.0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 7.0|
+---------+----------+-------------------+-------------+---------------+------+---------+------+-----+-----+-------+-------+-------+------+--------+-------------+-------------+
only showing top 5 rows

In [101]:
from pyspark.ml.regression import RandomForestRegressor

featuresCols = DF_model_actorsgt8.columns
featuresCols.remove('averageRating')
featuresCols.remove('tconst')

vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=2)

rf = RandomForestRegressor(labelCol="averageRating")

evaluator = RegressionEvaluator(metricName="rmse", labelCol=rf.getLabelCol(), predictionCol=rf.getPredictionCol())

paramGrid = ParamGridBuilder()\
                            .addGrid(rf.maxDepth, [2, 6]) \
                            .build()

cv = CrossValidator(estimator=rf, evaluator=evaluator, estimatorParamMaps = grid)

pipeline02 = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])

In [102]:
train_actorsgt8, test_actorsgt8 = DF_model_actorsgt8.randomSplit([0.7, 0.3], seed = 0)
pipelineModel02 = pipeline02.fit(train_actorsgt8.drop('tconst'))
predPipe02 = pipelineModel02.transform(test_actorsgt8.drop('tconst'))
evaluator = RegressionEvaluator(metricName="rmse")
RMSE_03 = evaluator.evaluate(predPipe02.selectExpr('features', 'averageRating as label', 'prediction'))
print("Model03: RMSE = " + str(round(RMSE_03,4)))

Model03: RMSE = 1.2204

In [103]:
Titanic_actorsgt8 = DF_model_actorsgt8.filter(DF_model_actorsgt8.tconst == 'tt0120338')
pred_Titanic_actorsgt8 = pipelineModel02.transform(Titanic_actorsgt8.drop('tconst'))
pred_Titanic_actorsgt8.selectExpr('features', 'averageRating as label', 'prediction').show(4)

+--------------------+-----+-----------------+
 features|label| prediction|
+--------------------+-----+-----------------+
(15,[0,1,2,3,8,11...| 7.8|6.486347808934546|
+--------------------+-----+-----------------+

**Q14 My final answer is :** <br/>
RandomForestRegressor was applied achieve this results and a variable with number of actors with Avg Rate 7 or higher.<br/>
Predicted Score for Titanic = 6.5 and Actual Score = 7.8 at the end the prediction has a error of 1.3. And almost the same RMSE, with RandomForest Regression RMSE = 1.22')